In [1]:
# This code cell is to get rid of annoying tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import numpy as np
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
from skimage import io, transform

In [3]:
train_images = glob.glob("train-images/*/*.png")
len(train_images)

187

In [4]:
input_size = 32
output_size = 128
scale_factor = 4

In [5]:
X_train = []
y_train = []

for img in train_images:
    img = io.imread(img)
    X_train.append(transform.resize(img, (input_size, input_size)))
    y_train.append(transform.resize(img, (output_size, output_size)))

X_train = np.array(X_train)
y_train = np.array(y_train)

In [6]:
X_train.shape, y_train.shape

((187, 32, 32, 3), (187, 128, 128, 3))

In [7]:
model = tf.keras.models.Sequential([
  #  Feature extraction
  tf.keras.layers.Conv2D(56,(5, 5), activation="relu", padding="same", input_shape=(input_size, input_size, 3)),
  #  Shrinking
  tf.keras.layers.Conv2D(56,(1, 1), activation="relu", padding="same"),
  #  Non-linear mapping
  tf.keras.layers.Conv2D(12,(3, 3), activation="relu", padding="same"),
  tf.keras.layers.Conv2D(12,(3, 3), activation="relu", padding="same"),
  tf.keras.layers.Conv2D(12,(3, 3), activation="relu", padding="same"),
  tf.keras.layers.Conv2D(12,(3, 3), activation="relu", padding="same"),
  #  Expanding:
  tf.keras.layers.Conv2D(56,(1, 1), activation="relu", padding="same"),
  #  Deconvolution:
  tf.keras.layers.Conv2DTranspose(3, (9, 9), strides=(scale_factor, scale_factor),  padding="same"),
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="mean_squared_error",
              metrics=['accuracy'])


In [8]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
6/6 [==============================] - 2s 205ms/step - loss: 0.2846 - accuracy: 0.3130
Epoch 2/10
6/6 [==============================] - 1s 210ms/step - loss: 0.1270 - accuracy: 0.3056
Epoch 3/10
6/6 [==============================] - 1s 199ms/step - loss: 0.0647 - accuracy: 0.5428
Epoch 4/10
6/6 [==============================] - 1s 197ms/step - loss: 0.0620 - accuracy: 0.5800
Epoch 5/10
6/6 [==============================] - 1s 196ms/step - loss: 0.0538 - accuracy: 0.5983
Epoch 6/10
6/6 [==============================] - 1s 208ms/step - loss: 0.0466 - accuracy: 0.6032
Epoch 7/10
6/6 [==============================] - 1s 211ms/step - loss: 0.0446 - accuracy: 0.5982
Epoch 8/10
6/6 [==============================] - 1s 198ms/step - loss: 0.0418 - accuracy: 0.5949
Epoch 9/10
6/6 [==============================] - 1s 210ms/step - loss: 0.0394 - accuracy: 0.6016
Epoch 10/10
6/6 [==============================] - 1s 206ms/step - loss: 0.0375 - accuracy: 0.6052


In [9]:
# model.save("models/model1")

In [ ]:
model = tf.keras.models.load_model("models/model1")

In [ ]:
test_images = glob.glob("test-images/*png")
len(test_images)

In [ ]:
fig = plt.figure(figsize=(5, 5))
columns = 2
rows = 2
for i in range(1, columns*rows + 1):
    lr_img = transform.resize(io.imread(test_images[i-1]), (input_size, input_size))
    fig.add_subplot(rows, columns, i)
    plt.imshow(lr_img)
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5))
columns = 2
rows = 2
for i in range(1, columns*rows + 1):
    lr_img = transform.resize(io.imread(test_images[i-1]), (input_size, input_size))
    hr_img_unshaped = model.predict(lr_img.reshape(-1, input_size, input_size, 3))
    hr_img = hr_img_unshaped.reshape(output_size, output_size, 3)
    fig.add_subplot(rows, columns, i)
    plt.imshow(hr_img)
plt.show()